In [2]:
# note Grakel does not seem to support Python >=3.10, Python 3.9 works fine
# you are free to remove imports that are not useful for you

#import sys
#!{sys.executable} -m pip install grakel
#!{sys.executable} -m pip install matplotlib
from grakel.datasets import fetch_dataset
from grakel.kernels import WeisfeilerLehman, VertexHistogram
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import KernelPCA # to check your own implementation
from sklearn.manifold import TSNE
import numpy as np
import scipy
import matplotlib.pyplot as plt
import math

In [19]:
# Some datasets, more datasets here https://ls11-www.cs.tu-dortmund.de/staff/morris/graphkerneldatasets
from ast import GtE
from grakel.kernels import ShortestPath
from grakel import Graph
"""
    The MUTAG dataset consists of 188 chemical compounds divided into two 
    classes according to their mutagenic effect on a bacterium. 

    The chemical data was obtained form http://cdb.ics.uci.edu and converted 
    to graphs, where vertices represent atoms and edges represent chemical 
    bonds. Explicit hydrogen atoms have been removed and vertices are labeled
    by atom type and edges by bond type (single, double, triple or aromatic).
    Chemical data was processed using the Chemistry Development Kit (v1.4).
"""

"""
    ENZYMES is a dataset of protein tertiary structures obtained from (Borgwardt et al., 2005) 
    consisting of 600 enzymes from the BRENDA enzyme database (Schomburg et al., 2004). 
    In this case the task is to correctly assign each enzyme to one of the 6 EC top-level 
    classes. 
"""

"""
    NCI1 and NCI109 represent two balanced subsets of datasets of chemical compounds screened 
    for activity against non-small cell lung cancer and ovarian cancer cell lines respectively
    (Wale and Karypis (2006) and http://pubchem.ncbi.nlm.nih.gov).
"""

dataset = fetch_dataset("NCI1", verbose=False) # just replace by the name of the datasets you want "ENZYMES", "NCI1"
G = dataset.data
y = dataset.target

wl_kernel = WeisfeilerLehman(n_iter=10, normalize=True, base_graph_kernel=VertexHistogram)
G_train, G_test, y_train, y_test = train_test_split(G, y, test_size=0.1, random_state=42)
#print(len(G_train))
#print(len(G_test))

K_train = wl_kernel.fit_transform(G)

print(""+ str(np.linalg.matrix_rank(K_train)))

'''K_test = wl_kernel.transform(G)

clf = SVC(kernel='precomputed')
clf.fit(K_train, y_train)
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='precomputed', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

y_pred = clf.predict(K_test)
print("%2.2f %%" %(round(accuracy_score(y_test, y_pred)*100)))'''

4002


'K_test = wl_kernel.transform(G)\n\nclf = SVC(kernel=\'precomputed\')\nclf.fit(K_train, y_train)\nSVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,\n    decision_function_shape=\'ovr\', degree=3, gamma=\'scale\',\n    kernel=\'precomputed\', max_iter=-1, probability=False, random_state=None,\n    shrinking=True, tol=0.001, verbose=False)\n\ny_pred = clf.predict(K_test)\nprint("%2.2f %%" %(round(accuracy_score(y_test, y_pred)*100)))'